In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

class CVRDataset(Dataset):
	def __init__(self, mode):
		self.mode = mode
		data_names = ["Sale","SalesAmountInEuro","Time_delay_for_conversion","click_timestamp","nb_clicks_1week","product_price",
				"product_age_group","device_type","audience_id","product_gender","product_brand","product_category1","product_category2",
				"product_category3","product_category4","product_category5","product_category6","product_category7","product_country",
				"product_id","product_title","partner_id","user_id"]
		string2num = ["product_age_group","device_type","product_gender","product_brand","product_category1","product_category2",
           "product_category3","product_category4","product_category5","product_category6","product_category7","product_country",
           "product_id","product_title","partner_id","user_id"]
		drop_data = ["SalesAmountInEuro", "click_timestamp", "audience_id","Time_delay_for_conversion"]
		self.data=pd.read_csv('/content/drive/MyDrive/연습/CriteoSearchData', delimiter='\t', names=data_names, nrows=1500000)

		# timestamp(1970.01.01 이후의 밀리초) to day, hour, minute
		self.data['click_timestamp'] = self.data["click_timestamp"].map(lambda x: int(x/1000)) # change second
		self.data['day'] = self.data["click_timestamp"].map(lambda x: int(x/86400)) # get day
		self.data['hour'] = self.data["click_timestamp"].map(lambda x: int(x%86400/3600)) # get hour
		self.data['minute'] = self.data["click_timestamp"].map(lambda x: int(x%86400%3600/60))  # get minute

		# drop unnecessary data
		for d in drop_data:
			self.data=self.data.drop(d, 1)

			
		# change string to label
		# TypeError: Encoders require their input to be uniformly strings or numbers. Got ['int', 'str']
		self.data['product_category7'] = str(self.data['product_category7'])
		for sn in string2num:
			le = LabelEncoder()
			self.data[sn]=le.fit_transform(self.data[sn])
   
		# normalize 0~1
		mms = MinMaxScaler()
		self.data[self.data.columns] = mms.fit_transform(self.data[self.data.columns])
  
		if self.mode =='Train':
			dstart, dend = 0, 1000000
		else:
			dstart, dend = 1000000, 1500000

		self.label, self.data = self.data['Sale'].values[dstart:dend], self.data.drop('Sale', 1).values[dstart:dend]
		self.data = torch.FloatTensor(self.data)
		self.label = torch.FloatTensor(self.label)

	def __len__(self):
		return self.data.shape[0]
  
	def __getitem__(self, idx):
		return self.data[idx], self.label[idx]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
